In [ ]:
# !pip install -U datasets
# !pip install transformers==4.38.2

In [ ]:
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset

## Utils

In [ ]:
class PromptTemplate:

    def __init__(self, developer_prompt, user_prompt_template):
        self.developer_prompt = developer_prompt
        self.user_prompt_template = user_prompt_template

    def format_user_prompt(self, text):
        return self.user_prompt_template.format(text=text)

# Loading Pre-Trained Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)


tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Loading Dataset

In [ ]:
dataset = load_dataset("openai/gsm8k", "main")

print(f'Dataset length : {len(dataset["test"])}')

test_samples = dataset["test"][:5]
questions = test_samples["question"]
long_answers = test_samples["answer"]
answers = [float(an.split("#### ")[-1]) for an in long_answers]

In [ ]:
df = pd.DataFrame({"questions":questions, "long_answers": long_answers, "answer": answers})

In [ ]:
df.head()

# Zero-Shot Prompting

In [ ]:
developer_prompt = """You are a helpful AI assistant who knows math."""
user_prompt = """Below I will provide a question with a math problem.
Please solve it and present final number which is an answer to the problem.
Do not show any explanation and do not provide units.

Question: {text}
Give answer in this form: {{"answer": "answer with final number"}}"""

prompt_template = PromptTemplate(developer_prompt, user_prompt)
developer_prompt = prompt_template.developer_prompt

In [ ]:
messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": user_prompt},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0
}

In [ ]:
for index, question in tqdm(enumerate(df.questions.iloc[:5])):
    user_prompt = prompt_template.format_user_prompt(question)
    messages = [
        {"role": "developer", "content": developer_prompt},
        {"role": "user", "content": user_prompt},
    ]
    output = pipe(messages, **generation_args)
    print("Raw output:", output[0]["generated_text"])

# Zero Shot with Chain of Thought Prompting

In [ ]:
# Usage Example
developer_prompt = "You are a helpful AI assistant who knows math."
user_prompt_template = """Below I will provide a question with a math problem.
Please solve it and present the final number which is the answer to the problem.
In the final answer do not provide units, give only the number.

Question: {text}
Give answer in this form: {{"reasoning": "Solve it step by step and provide reasoning and explanation", \n "answer": "final number"}}"""

prompt_template = PromptTemplate(developer_prompt, user_prompt_template)

In [ ]:
for index, question in tqdm(enumerate(df.questions.iloc[:5])):
    user_prompt = prompt_template.format_user_prompt(question)
    messages = [
        {"role": "developer", "content": developer_prompt},
        {"role": "user", "content": user_prompt},
    ]
    output = pipe(messages, **generation_args)

    print("Raw output:", output[0]["generated_text"])